In [1]:
from pyspark.sql import SparkSession


# Initialize PySpark with MongoDB support
APP_NAME = "Collecting and Displaying Records"
spark = (
    SparkSession.builder.appName(APP_NAME)
    # Load support for MongoDB and Elasticsearch
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.elasticsearch:elasticsearch-spark-30_2.12:7.14.2")
    # Add Configuration for MongopDB
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/test.coll")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/test.coll")

    .getOrCreate()
)
sc = spark.sparkContext
sc.setLogLevel("ERROR")

print("\nPySpark initialized...")

22/02/17 21:26:54 WARN Utils: Your hostname, kellyui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
22/02/17 21:26:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/kelly/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kelly/.ivy2/cache
The jars for the packages stored in: /Users/kelly/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebcd3afe-1488-4349-8f5e-23d3d76f4ce1;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.14.2 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
:: resolution repo


PySpark initialized...


In [3]:
on_time_df = spark.read.parquet('data/on_time_performance.parquet')

# Use SQL to look at the total flights by month across 2015
on_time_df.createOrReplaceTempView("on_time_dataframe")
total_flights_by_month = spark.sql(
  """SELECT INT(Month), INT(Year), COUNT(*) AS total_flights
  FROM on_time_dataframe
  GROUP BY INT(Year), INT(Month)
  ORDER BY INT(Year), INT(Month)"""
)

total_flights_by_month.toPandas()

,Month,Year,total_flights
0,1,2015,469968
1,2,2015,429191
2,3,2015,504312
3,4,2015,485151
4,5,2015,496993
5,6,2015,503897
6,7,2015,520718
7,8,2015,510536
8,9,2015,464946
9,10,2015,486165


In [4]:
# Save chart to MongoDB
(
    total_flights_by_month.write.format("mongo")
    .mode("append")
    .option("database", "agile_data_science")
    .option("collection", "flights_by_month")
    .save()
)

print("Wrote to MongoDB!")

22/02/17 21:29:48 ERROR Executor: Exception in task 0.0 in stage 16.0 (TID 14)1]
com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=mongo:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketException: mongo}, caused by {java.net.UnknownHostException: mongo}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:177)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:147)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:98)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:278)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.execute(MongoClie

Py4JJavaError: An error occurred while calling o63.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 14) (192.168.0.3 executor driver): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=mongo:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketException: mongo}, caused by {java.net.UnknownHostException: mongo}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:177)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:147)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:98)
	at com.mongodb.client.internal.MongoClientDelega

In [5]:
! mongosh mongodb://mongo:27017/agile_data_science --eval 'db.flights_by_month.find().sort({"Year": 1, "Month": 1})'

]0;mongosh mongodb://mongo:27017/agile_data_science?directConnection=trueCurrent Mongosh Log ID:	620e403bda778c4616a89587
Connecting to:		mongodb://mongo:27017/agile_data_science?directConnection=true&appName=mongosh+1.1.9
MongoNetworkError: getaddrinfo ENOTFOUND mongo


In [ ]:
on_time_df.columns